In [ ]:
import pandas as pd
import sys
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
print(widgets.__version__)

In [ ]:
%matplotlib inline

In [ ]:
use_date = widgets.Checkbox(
    description='Date ',
    value=True,
    indent=True,
    style={'description_width': '20%'},
    layout={'border': '1px solid black', 'width':'100px'},
)
use_date

In [ ]:
btn = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn.layout.object_position = 'right'
btn.layout.border= '1px solid black'
btn.layout.margin= '10px 0px 10px 100px'
btn

In [ ]:
widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

In [ ]:
tab = widgets.Tab()    
out = widgets.Output(layout={'border': '1px solid black'})
up = widgets.FileUpload(accept="", multiple=False)

In [ ]:
delim = widgets.RadioButtons(
    options=[';', ',', ' '],
    description='Separator: ',
    disabled=False)

eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)

eraser

In [ ]:
rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
rows

In [ ]:
button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to Upload',
    icon='check')
button_upload 

In [ ]:
button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')
button_preview 

In [ ]:
button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='danger',
    tooltip='Click to Plot',
    icon='pencil')
button_plot

In [ ]:
graph_type = widgets.Dropdown(
    options=['Bar Chart', 'Line Chart'],
    value='Bar Chart',
    description='Chart Type:',
    disabled=False)
graph_type

In [ ]:
x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)
x_axis

In [ ]:
y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)
y_axis

In [ ]:
color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='lightblue',
    disabled=False)
color_picker

In [ ]:
toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])
toggle

In [ ]:
accordion = widgets.Accordion(children=[
    widgets.HBox([
                widgets.BoundedIntText(description='aaaaa', layout={'max_width': '290px', 'width':'600px'}),
                widgets.BoundedIntText(description='aaaaa', layout={'max_width': '290px'}),
                widgets.BoundedIntText(description='aaaaa', layout={'max_width': '290px'}),
                widgets.BoundedIntText(description='aaaaa', layout={'max_width': '290px'})],
        layout=widgets.Layout(flex_flow='row wrap')),
    up,
    widgets.VBox([delim, eraser]), 
    rows,
        ],
    layout = widgets.Layout(padding='21px', border='solid')
    )
accordion.set_title(0, 'Test')
accordion.set_title(1, 'File Selection')
accordion.set_title(2, 'Delimiter')
accordion.set_title(3, 'Skip Rows')
accordion

In [ ]:
accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview, button_upload]),
    out
])
accordion_box

In [ ]:
children = [
    accordion_box, 
    widgets.VBox([toggle, out]),
    widgets.VBox([
        widgets.HBox([graph_type, color_picker]),
        widgets.HBox([x_axis, y_axis]), 
        button_plot,
        out
])]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Describer")
tab.set_title(2, "Plotter")

In [ ]:
def content_parser():
    if up.value == {}:
        with out:
            print('No CSV loaded')    
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            if typ == "text/csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')
    
            if eraser.value != {}: 
                for val in eraser.value:
                    if val == "tab":
                        content_str = content_str.replace("\t","")
                    else:
                        content_str = content_str.replace(val,"")
            if content_str != "":
                str_io = StringIO(content_str) 
                return str_io
def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value)    
        return df
    else:
        return None
def preview():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n -----Now this is how your DF looks like:----- \n')
        if df is not None:
            print(df.head(10))
        else:
            print('Configuration is wrong/missing...')
def upload():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n --------Your uploaded DF looks like:-------- \n')
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
        else:
            print('Configuration is wrong/missing...')
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats  ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    print('Configuration is wrong/missing...')
    
    
def plot():
    graph = graph_type.value
    if graph != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                graph))
            if (df is not None):
                df = df.head(5)
                height = df[y_axis.value]
                bars = df[x_axis.value]
                y_pos = np.arange(len(height))
                plt.figure(figsize=(10,4))
                if graph == 'Bar Chart':
                    plt.bar(
                        y_pos, 
                        height, 
                        color=color_picker.value)
                    plt.xticks(y_pos, bars)
                elif graph == 'Line Chart':
                    plt.plot(
                        bars,
                        height, 
                        color=color_picker.value,
                        marker='o', 
                        linestyle='solid'
                    )
                    plt.xticks(bars)
                plt.show()
                

In [ ]:
def preview_clicked(b):
    preview()
def upload_clicked(b):
    upload()
def desc_clicked(b):
    desc()
def plotter_clicked(b):
    plot()
    
button_preview.on_click(preview_clicked)
button_upload.on_click(upload_clicked)
toggle.observe(desc_clicked, 'value')
button_plot.on_click(plotter_clicked)

In [ ]:
tab

In [ ]:
# can select multiple
toggle_buttons= {'a':widgets.ToggleButton(description='a', value=False),
          'b':widgets.ToggleButton(description='b', value=False),
          'c':widgets.ToggleButton(description='c', value=False),
          'd':widgets.ToggleButton(description='d', value=False)}

toggle_disp = widgets.VBox([
                widgets.HBox([toggle_buttons['a'], toggle_buttons['b']], description='htest'),
                widgets.HBox([toggle_buttons['c'], toggle_buttons['d']])
                            ], description='test')

def on_toggle(**toggles):
    for name, obj in toggles.items():
        print(name, obj)
    # print(toggles)

interact_out = widgets.interactive_output(on_toggle, toggle_buttons)
display(toggle_disp, interact_out)

In [ ]:
# intext = widgets.IntText(min=1, max=100, step=1, continuous_update=True)
# 
# widgets.jslink((tb_fruits, 'value'), (intext, 'value'))
# widgets.VBox([play, slider, out])

In [ ]:
# only one
fruits = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
tb_fruits = widgets.ToggleButtons(options = fruits, style={'button_width':'100px'}, layout={'width': '320px'})
print(tb_fruits.value)
tb_fruits 

In [ ]:
tb = widgets.ToggleButtons(
    options=[('Slow', '11'), ('Regular', '22'), ('Fast', '33')],
    description='Speed:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    value = '11'
#     icons=['check'] * 3
)
print(tb.value)
tb